In [1]:
import os

In [2]:
base_dir = './'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [3]:
train_normal_dir = os.path.join(train_dir, 'Normal')
train_under_dir = os.path.join(train_dir, 'UnderExposed')
train_over_dir = os.path.join(train_dir, 'OverExposed')

validation_normal_dir = os.path.join(validation_dir, 'Normal')
validation_under_dir = os.path.join(validation_dir, 'UnderExposed')
validation_over_dir = os.path.join(validation_dir, 'OverExposed')

In [4]:
print('total training normal images:', len(os.listdir(train_normal_dir)))
print('total training underexposed images:', len(os.listdir(train_under_dir)))
print('total training overexposed images:', len(os.listdir(train_over_dir)))

print('total validation normal images:', len(os.listdir(validation_normal_dir)))
print('total validation underexposed images:', len(os.listdir(validation_under_dir)))
print('total validation underexposed images:', len(os.listdir(validation_over_dir)))

total training normal images: 3001
total training underexposed images: 6001
total training overexposed images: 6035
total validation normal images: 534
total validation underexposed images: 1069
total validation underexposed images: 1035


In [5]:
from tensorflow.keras import layers
from tensorflow.keras import Model

2022-01-12 22:50:16.147992: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-12 22:50:16.148089: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

2022-01-12 22:50:52.000801: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-12 22:50:52.000832: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-12 22:50:52.000854: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (faiz-X556UAM): /proc/driver/nvidia/version does not exist
2022-01-12 22:50:52.001151: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)

2022-01-12 22:50:52.990455: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 37879808 exceeds 10% of free system memory.
2022-01-12 22:50:54.386935: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 37879808 exceeds 10% of free system memory.
2022-01-12 22:50:54.407473: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 37879808 exceeds 10% of free system memory.


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0     

In [24]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

/home/faiz/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 15037 images belonging to 3 classes.
Found 2638 images belonging to 3 classes.


In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

/home/faiz/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-01-12 00:53:50.660057: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
100/100 - 163s - loss: -1.5752e+05 - acc: 0.4170 - val_loss: -7.9192e+05 - val_acc: 0.4090
Epoch 2/15
100/100 - 202s - loss: -5.3570e+06 - acc: 0.4200 - val_loss: -1.1844e+07 - val_acc: 0.3830
Epoch 3/15
100/100 - 119s - loss: -5.4006e+07 - acc: 0.4050 - val_loss: -7.6794e+07 - val_acc: 0.3890
Epoch 4/15
100/100 - 133s - loss: -1.6753e+08 - acc: 0.4010 - val_loss: -2.7134e+08 - val_acc: 0.3970
Epoch 5/15
100/100 - 102s - loss: -3.8463e+08 - acc: 0.3860 - val_loss: -5.5708e+08 - val_acc: 0.3900
Epoch 6/15
100/100 - 130s - loss: -9.2267e+08 - acc: 0.3995 - val_loss: -1.3091e+09 - val_acc: 0.3870
